# 4.7 Elgamal Cryptosystem

Bob:
- $p$: prime
- $g$: primitive root of $p$
- $0 \leq x < p - 1$: private key
- $h = g^x$ mod $p$ 
- ($p, g, h$): public key

> Hard to find $x = log_g(h \text{ mod } p)$ for large values of $h$

---

Alice:
- $0 \leq m < p - 1$: plaintext message
- $0 \leq y < p - 1$: ephemeral key
    - unique $y$ per $m$ (hence ephemeral, short-lived and created a lot)
    - Bob doesn't need to know ephemeral key
- $s = h^y$ mod $p$
    - $c_1 = g^y$ mod $p$ 
    - $c_2 = ms$ (mod $p$)
    > Note: $s$ and $c_1$ can quickly be computed w/ binary exponentiation
- ($c_1, c_2$): ciphertext

--- 

Bob:
$$
c^x_1 = (g^y)^x = g^{xy} = (g^x)^y \equiv h^y \equiv s\text{ mod } p
$$

> Note: Bob found $s$ without knowing ephemeral key $y$. Can use extended euclidean algo (egcd) to find $(c_1^x)^{-1}$ mod $p$

$$
c_2(c_1^x)^{-1} \equiv c_2s^{-1} \equiv (ms)s^{-1} \equiv m\cdot 1 =m, \text{\qquad (mod $p$)} 
$$

## Why Elgamal is prob secure (for now...)

2 main strategies that Eve might use:

### Find x

Can hijack the decryption algorithm,

BUT need to solve discrete log base $g$ of $h$ mod $p$. 

$$ x = log_g(h \text{ mod } p) $$

### Find y

Can compute $h^y$, then $(h^y)^{-1} \equiv s^{-1}$ (mod $p$). Then can hijack the second half of the decryption algorithm,

BUT need to solve discrete log base $g$ of $c_1$ mod $p$.

$$ y = log_g(c_1 \text{ mod } p) $$

### Discrete Logarithm Problem (4.7.4)

> Suppose you are given a prime $p$, a primitive root $g$ mod $p$, and an integer $a$ co-prime to $p$. Find the discrete log base $g$ of $a$ mod $n$. I.e. find the unique integer $k$ such that $0 \leq k \leq p-1$ such that $g^k \equiv a$ (mod $p$)

As $p$ gets large, classical computers struggle.

There are no known algorithms that accomplish this task polynomially in number of digits of $p$

## Code Examples

In [7]:
p = 4115549
g = 2
x = 2634326     # PrivKey

h = pow(g, x, p)

(p, g, h)       # PubKey

(4115549, 2, 1149114)

In [8]:
m = 3340481
y = 2775147         # EphKey

s = pow(h, y, p)    # Secret
c_1 = pow(g, y, p)
c_2 = s*m % p

(c_1, c_2)

(621674, 1911501)

In [15]:
cx_1 =pow(c_1, x, p)
s_inv = pow(cx_1, -1, p)

c_2*s_inv % p

3340481

In [36]:
from random import randint

# Generates an Elgamal (pubkey, privkey) pair
# p:        prime
# g:        primitive root of p
#
# (opt) x:  Private key
#           can be provided otherwise randomly generated within [0, p-1) 
#
# return:   ((p,g,h), x) pubkey,privkey pair
def gen_elg(p, g, x=randint(0, p-2)):
    h = pow(g, x, p)
    return ((p, g, h), x)

In [35]:
# Performs Elgamal encryption 
# m:        message assumed within (0, p-1) range
# pubkey:   (p, g, h) pubkey generated by gen_elg
# 
# (opt) y:  Ephemeral key 
#           can be provided otherwise randomly generated within (0, p-1)
# 
# returns:  (c_1, c_2) ciphertext
def elg_enc(m, pubkey, y=randint(1, p-2)):
    p, g, h = pubkey

    s = pow(h, y, p)

    c_1 = pow(g, y, p)
    c_2 = s * m % p

    return (c_1, c_2)

# Performs Elgamal decryption
# c:        (c_1, c_2) ciphertext from elg_enc
# privkey:  (x) privkey generated by gen_elg
# 
# returns:  m decrypted from ciphertext (c) with privkey (k)
def elg_dec(c, privkey):
    x = privkey

    c_1, c_2 = c
    cx_1 = pow(c_1, x, p)

    s_inv = pow(cx_1, -1, p)
    return c_2 * s_inv % p

In [39]:
pubkey, privkey = gen_elg(4115549, 2, x=2634326)
print("pubkey: ", pubkey, "\nprivkey: ", privkey)

c = elg_enc(3340481, pubkey, y=2775147)
print("ciphertext: ", c)

m = elg_dec(c, privkey)
print("plaintext: ", m)

pubkey:  (4115549, 2, 1149114) 
privkey:  2634326
ciphertext:  (621674, 1911501)
plaintext:  3340481


# 4.8 Diffie-Hellman Key Exchange

Not quite a cryptosystem for message exhange.

More of a protocol to allow a shared secret without either having full control over content of shared secret.

Application-wise, this shared secret can be used as the key for a symmetric key cipher like a one-time pad.

---

Alice & Bob:
- $p$: fixed prime
- $g$: primitive root mod $p$

Alice:
- $0 \leq a < p - 1$
- $x = g^a$ mod $p$ ---> Bob

Bob:
- $0 \leq b < p - 1$
- $y = g^b$ mod $p$ ---> Alice

$$
y^a \equiv (g^b)^a = g^{ab} = (g^a)^b \equiv x^b \text{\qquad (mod $p$)}
$$